In [1]:
# Requests for handling HTTP get and other requests
import requests
import time # import for playing nice and not getting blocked
import pandas as pd
# from BeautifulSoup4 import BeatifulSoup if installed through pip install BeautifulSoup4
# 
from bs4 import BeautifulSoup

In [ ]:
# if bs4 not found install it with
#  !pip install beatifulsoup4
# https://www.crummy.com/software/BeautifulSoup/

In [2]:
centrs = "https://www.ss.com/lv/real-estate/flats/riga/centre/sell/"

In [3]:
req = requests.get(centrs)
req.status_code

200

In [4]:
soup = BeautifulSoup(req.text, 'lxml') # lxml is improved parser, a little bit more lenient
soup.title

<title>SS.COM Dzīvokļi - Rīga - Centrs, Cenas, Pārdod - Sludinājumi</title>

In [55]:
type(soup)

bs4.BeautifulSoup

In [5]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
tables = soup.find_all("table")
len(tables) 

7

In [6]:

headline = soup.find("tr", {"id":"head_line"}) # this is a shorter way of finding by tr AND this element having particular id
headline

<tr id="head_line">
<td class="msg_column" colspan="3" width="70%">
<span style="float:left;"> Sludinājumi
</span>
<span align="right" class="msg_column" style="float:right;text-align:right;padding-right:3px;">
<noindex>
<a class="a19" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4S.html" rel="nofollow">datums</a></noindex></span>
</td>
<td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SFDwT.html" rel="nofollow" title="">Iela</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SelM=.html" rel="nofollow" title="">Ist.</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4QelM=.html" rel="nofollow" title="">m2</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4XelM=.html" rel="nofollow" title

In [56]:
type(headline)

bs4.element.Tag

In [7]:
headtds = headline.find_all("td")
# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/td
headtds

[<td class="msg_column" colspan="3" width="70%">
 <span style="float:left;"> Sludinājumi
 </span>
 <span align="right" class="msg_column" style="float:right;text-align:right;padding-right:3px;">
 <noindex>
 <a class="a19" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4S.html" rel="nofollow">datums</a></noindex></span>
 </td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SFDwT.html" rel="nofollow" title="">Iela</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4SelM=.html" rel="nofollow" title="">Ist.</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4QelM=.html" rel="nofollow" title="">m2</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/lv/real-estate/flats/riga/centre/sell/fDgSeF4XelM=.html" rel="nofollow" title="

In [8]:
headcolumns = [el.text for el in headtds[1:]] #.text gets us content even from children and grandchildren
headcolumns

['Iela', 'Ist.', 'm2', 'Stāvs', 'Sērija', 'Cena, m2', 'Cena']

In [9]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [10]:
column_names = getColList(soup)
column_names

['description',
 'url',
 'Iela',
 'Ist.',
 'm2',
 'Stāvs',
 'Sērija',
 'Cena, m2',
 'Cena']

In [11]:
trows = soup.find_all('tr')
len(trows)


39

In [12]:
# hardest part in this 
# how to filter only specific rows
# we need to find something that is unique to our apartment rows but not to the extra rows
# remember we id is not guaranteed
apt_rows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
len(apt_rows)

30

In [13]:
apt_rows[-1]

<tr id="tr_47585624"><td class="msga2 pp0"><input id="c47585624" name="mid[]" type="checkbox" value="47585624_1106_0"/></td><td class="msga2"><a href="/msg/lv/real-estate/flats/riga/centre/elxpc.html" id="im47585624"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/4/637/159119/31823755.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="d2MlOTglOUYlN0UlQTUlQ0MlODZ5aSU5OCU5RnclQTYlODclOER1aCU5OCU5OHUlQTYlODAlODV0|D3bfErPU" href="/msg/lv/real-estate/flats/riga/centre/elxpc.html" id="dm_47585624">Autostāvvieta iekļauta cenā 8000 euro vērtībā

Piedāvājumā lie</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Ģertrūdes 99</td><td c="1" class="msga2-o pp6" nowrap="">3</td><td c="1" class="msga2-o pp6" nowrap="">57</td><td c="1" class="msga2-o pp6" nowrap="">2/6</td><td c="1" class="msga2-o pp6" nowrap="">Renov.</td><td c="1" class="msga2-o pp6" nowrap="">1,979 €</td><td c="1" class="msga2-o pp6" nowrap="">112,800  €</td></tr>

In [14]:
# lets make a function from the above doodle and make it work on most pages on SS
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [15]:
row_tds = apt_rows[0].find_all('td')
len(row_tds)

10

In [17]:
row_tds[1].text

''

In [18]:
row_tds[2].text

'Piedāvājam iegādāties dzīvokli projektā San Antonia, kas atrodas'

In [19]:
row_tds[-1].text

'150,000  €'

In [20]:
a = row_tds[1].find('a')
a

<a href="/msg/lv/real-estate/flats/riga/centre/cnpxe.html" id="im48453836"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/3/545/136236/27247153.th2.jpg"/></a>

In [21]:
a.attrs

{'href': '/msg/lv/real-estate/flats/riga/centre/cnpxe.html',
 'id': 'im48453836'}

In [22]:
a.get('href')

'/msg/lv/real-estate/flats/riga/centre/cnpxe.html'

In [23]:
def getRow(row,colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) <3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [24]:
getRow(apt_rows[0])

{'description': 'Piedāvājam iegādāties dzīvokli projektā San Antonia, kas atrodas',
 'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/cnpxe.html',
 'Iela': 'Antonijas 11',
 'Ist.': '2',
 'm2': '58',
 'Stāvs': '2/6',
 'Sērija': 'Renov.',
 'Cena, m2': '2,586 €',
 'Cena': '150,000  €'}

In [25]:
def getRows(rowlist,colist=column_names):
    return [getRow(row) for row in rowlist]


In [26]:
row_ads = getRows(apt_rows)
row_ads[-3:]

[{'description': 'Предлагаем к продаже шикарный, современный пентхаус с эксклюзивн',
  'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/apmed.html',
  'Iela': 'Grostonas 21',
  'Ist.': '4',
  'm2': '140',
  'Stāvs': '10/10',
  'Sērija': 'Jaun.',
  'Cena, m2': '2,679 €',
  'Cena': '375,000  €'},
 {'description': 'Dzīvoklis Klusajā centrā. \r\nSaulains divistabu dzīvoklis Ausekļa',
  'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/adfbj.html',
  'Iela': 'Ausekļa 3',
  'Ist.': '2',
  'm2': '52',
  'Stāvs': '5/5',
  'Sērija': 'P. kara',
  'Cena, m2': '1,923 €',
  'Cena': '100,000  €'},
 {'description': 'Autostāvvieta iekļauta cenā 8000 euro vērtībā\r\n\r\nPiedāvājumā lie',
  'url': 'https://ss.com/msg/lv/real-estate/flats/riga/centre/elxpc.html',
  'Iela': 'Ģertrūdes 99',
  'Ist.': '3',
  'm2': '57',
  'Stāvs': '2/6',
  'Sērija': 'Renov.',
  'Cena, m2': '1,979 €',
  'Cena': '112,800  €'}]

In [31]:
dtemp = pd.DataFrame(row_ads, columns=column_names)
dtemp.shape

(30, 9)

In [27]:
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist)
    return pd.DataFrame(rows, columns=column_names)
    

In [28]:
df = getDFfromURL(centrs)

In [29]:
df.shape

(30, 9)

In [30]:
df.head()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Piedāvājam iegādāties dzīvokli projektā San An...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Antonijas 11,2,58,2/6,Renov.,"2,586 €","150,000 €"
1,"Pārdod 2 istabu dzīvokli Zirņu ielā 6, Rīgā (b...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Zirņu 6,2,44,3/6,Hrušč.,"1,477 €","65,000 €"
2,"Pārdošanā 2 istabu dzīvoklis, pirmskara laika ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Tomsona 22,2,59,3/5,P. kara,"1,441 €","85,000 €"
3,Autostāvvieta iekļauta cenā ( 8000€ vērtībā)\r...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 99,3,67,4/6,P. kara,"1,684 €","112,800 €"
4,Ģimene pārdod mājīgu un gaišu 3 istabu dzīvokl...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 135,3,70,5/5,P. kara,907 €,"63,500 €"


In [32]:

# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/a
anchors = soup.find_all("a")
len(anchors)

98

In [33]:
anchors[:5]

[<a href="/" title="Sludinājumi"><img alt="Sludinājumi" border="0" class="page_header_logo" src="https://i.ss.com/img/p.gif"/></a>,
 <a class="a_menu" href="/lv/real-estate/flats/new/" title="Iesniegt Sludinājumu">Iesniegt Sludinājumu</a>,
 <a class="a_menu" href="/lv/login/" title="Mani Sludinājumi">Mani Sludinājumi</a>,
 <a class="a_menu" href="/lv/real-estate/flats/riga/centre/search/" title="Meklēt sludinājumus">Meklēšana</a>,
 <a class="a_menu" href="/lv/favorites/" title="Memo">Memo</a>]

In [34]:
navs = [anchor for anchor in anchors if anchor.get("name") == "nav_id"] # get will get us None if no "name" exists
len(navs)

10

In [37]:
navs[0]

<a class="navi" href="/lv/real-estate/flats/riga/centre/sell/page27.html" name="nav_id" rel="prev"><img border="0" height="5" src="https://i.ss.com/img/s_left.png" style="padding-bottom:2px;" width="9"/> Iepriekšējie</a>

In [38]:
navs[0].attrs

{'name': 'nav_id',
 'rel': ['prev'],
 'class': ['navi'],
 'href': '/lv/real-estate/flats/riga/centre/sell/page27.html'}

In [39]:
navs[0]['href']

'/lv/real-estate/flats/riga/centre/sell/page27.html'

In [40]:
# could use regular expression here but we can use split 
afterpage = navs[0]['href'].split("/page")[1]
afterpage

'27.html'

In [41]:
beforedot = afterpage.split(".html")[0]
beforedot

'27'

In [42]:
lastpage = int(beforedot)
lastpage

27

In [45]:
def getAllLocalUrls(url):
    """Get a list of all urls including paginated pages"""
    results = [url] # default is just the url if no extra pages found
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    # we just need a one element
    prevanchor = soup.find('a', {"rel":"prev"})
    if prevanchor == None:
        return results
    href = prevanchor.attrs.get('href')
    lastPageNum = int(href.split('page')[-1].split('.')[0])
    print("Last page is",lastPageNum)
    nurls = [f"{url}page{n}.html" for n in range(2,lastPageNum+1)]
    results += nurls
    return results

In [46]:
local_urls = getAllLocalUrls(centrs)
len(local_urls),local_urls[:3],local_urls[-1]

Last page is 27


(27,
 ['https://www.ss.com/lv/real-estate/flats/riga/centre/sell/',
  'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page2.html',
  'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page3.html'],
 'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page27.html')

In [47]:
dlast = getDFfromURL(local_urls[-1])
dlast.shape

(7, 9)

In [48]:
dlast.head()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Īpašnieks pārdod 1-istabas dzīvokli renovētā p...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Miera 5,1,35,1/7,Renov.,"1,500 €","52,500 €"
1,Pārdod remontējamu dzīvokli. Dzīvoklis sastāv ...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ieroču 10,3,57,5/6,P. kara,930 €,"53,000 €"
2,Продаёт владелец. Апартаменты для настоящих лю...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Skolas 20,3,110,2/6,Jaun.,"2,909 €","320,000 €"
3,Skaists un gaumīgs 3-istabu dzīvoklis renovētā...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Klusā 11,3,66,2/5,Renov.,"2,210 €","145,860 €"
4,Стильная и комфортная квартира в современном п...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Vesetas 15,3,109,3/5,Jaun.,"3,028 €","330,000 €"


In [49]:
def get_all_ads_df(start_url, save_excel_path=None):
    df_list=[]
    local_urls = getAllLocalUrls(start_url)
    for url in local_urls:
        print(f"Gathering data from {url}")
        df_list.append(getDFfromURL(url))
        time.sleep(0.3) # we need this to play nice!
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
    big_df = pd.concat(df_list)
    if save_excel_path:
        big_df.to_excel(save_excel_path)
    return big_df
    

In [50]:
centrs

'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/'

In [51]:
d = get_all_ads_df(centrs, "centrs.xlsx")

Last page is 27
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page2.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page3.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page4.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page5.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page6.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page7.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page8.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page9.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page10.html
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/centre/sell/page11.html
Gathering data from http

In [52]:
d.shape

(787, 9)

In [53]:
d.head()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Piedāvājam iegādāties dzīvokli projektā San An...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Antonijas 11,2,58,2/6,Renov.,"2,586 €","150,000 €"
1,"Pārdod 2 istabu dzīvokli Zirņu ielā 6, Rīgā (b...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Zirņu 6,2,44,3/6,Hrušč.,"1,477 €","65,000 €"
2,"Pārdošanā 2 istabu dzīvoklis, pirmskara laika ...",https://ss.com/msg/lv/real-estate/flats/riga/c...,Tomsona 22,2,59,3/5,P. kara,"1,441 €","85,000 €"
3,Autostāvvieta iekļauta cenā ( 8000€ vērtībā)\r...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 99,3,67,4/6,P. kara,"1,684 €","112,800 €"
4,Ģimene pārdod mājīgu un gaišu 3 istabu dzīvokl...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Ģertrūdes 135,3,70,5/5,P. kara,907 €,"63,500 €"


In [54]:
d.tail()

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
2,Продаёт владелец. Апартаменты для настоящих лю...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Skolas 20,3,110,2/6,Jaun.,"2,909 €","320,000 €"
3,Skaists un gaumīgs 3-istabu dzīvoklis renovētā...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Klusā 11,3,66,2/5,Renov.,"2,210 €","145,860 €"
4,Стильная и комфортная квартира в современном п...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Vesetas 15,3,109,3/5,Jaun.,"3,028 €","330,000 €"
5,Projekta Parker's attīstītājs pārdod trīs ista...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Valdemāra 41,3,80,1/7,Jaun.,"3,328 €","266,200 €"
6,Piedāvājumā 4 istabu dzīvoklis vēsturiskā namā...,https://ss.com/msg/lv/real-estate/flats/riga/c...,Čaka 36,4,84,3/5,P. kara,"1,821 €","152,935 €"


# TODO 
# Try with different starting address not only centrs
## Maybe combine regions
## See how it would work with maybe cars
## Data engineering make new columns based on existing ones, clean some columns
### Changing floors 2/6 to columns 2 and 6, clean up Euro signs
## Sorting, Describing, Grouping by regions etc